# 第一个版本的MHA

In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import math

In [2]:
# 输入模型超参数
batch_model = 2
time_model = 3
d_model = 4
n_head = 4

In [3]:
# 随机创建输入, qkv
X = torch.randn(batch_model, time_model, d_model)
X

tensor([[[-0.6769,  0.6549, -0.0496, -1.5999],
         [-1.3586, -0.6275, -0.9502, -0.3471],
         [-0.6865,  0.1135,  0.9351,  0.1410]],

        [[-0.1647,  0.2453,  0.1300, -0.8140],
         [ 0.1070,  1.3536, -1.1907,  1.2817],
         [ 0.8633, -0.0731,  0.0484, -1.6660]]])

In [4]:
# 创建模型结构
# 问题1：前面这一这段代码的含义是什么，python相关
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head):
        # 注意这里super要传入自己和self，不能只传入自己，你要多学一些python的基础知识了
        super(MultiHeadAttention, self).__init__()
        
        # 初始化超参数
        self.n_head = n_head
        self.d_model = d_model
        # 创建qkv的三个线性映射层，用一个矩阵和qkv相乘，让qkv变得可学习
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        # 问题2：这个combine的作用是什么，矩阵经过怎样的运算
        self.combine = nn.Linear(d_model,d_model)
        self.softmax = nn.Softmax(dim=-2)

    def forward(self, q, k, v):
        batch, time, dimension = q.shape
        # 得到每个头的dimension
        n_d = d_model // n_head
        # 让qkv进入线性层
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        # 拆分qkv
        # 问题3:为什么需要permute这个东西： 
        # 问题4:这个转置函数怎么用的，结果是什么
        # 问题5:这两个四维矩阵是怎么乘起来的
        # 为了更好的并行计算，变换之后，qkv矩阵形状变成(batch, n_head, time, n_d)，之后k要再次变换成(batch, time, m_head, n_d)
        # 最后q @ k结果的shape是(batch, n_head, time, time)两个四维矩阵相乘，前面两个维度都不参与运算。
        q = q.view(batch, time, n_head, n_d).permute(0, 2, 1, 3)
        k = k.view(batch, time, n_head, n_d).permute(0, 2, 1, 3)
        v = v.view(batch, time, n_head, n_d).permute(0, 2, 1, 3)
        # 都是交换维度，只是交换维度的数量区别
        print(k.shape)
        print('------------')
        print(k.permute(0, 1, 3, 2).shape)
        print(k.transpose(2, 3).shape)
        # 计算注意力结果 q*k转置/n_d
        # score的shape为(batch, n_head, time, time)
        score = q @ k.transpose(2,3) / math.sqrt(n_d)
        # 生成mask，一个左下角为1的下三角矩阵
        mask = torch.tril(torch.ones(time, time, dtype=bool))
        # 让mask掩盖score中不应该被注意的数值,全部变成负无穷
        # 问题6：这个masked_fill函数是哪里的，矩阵自带的吗，怎么用， pytorch张量自带的方法，用于用特定值填充掩码位置。
        # 为什么需要combine
        # 在多头注意力机制中，将多个头的输出重新组合成一个张量后，还需要通过一个线性变换来生成最终的输出。这是因为每个头的输出仅仅代表该头的注意力结果，最后通过线性层来融合各个头的信息，生成最终的表示。
        # 1. 线性变换：尽管 permute 和 view 将多个头的输出重新组合成了一个张量，但是每个头的输出还需要进一步线性变换，以便在融合信息的同时对其进行适当的缩放和变换。
        # 2. 参数化：combine 层通过线性层实现参数化，使得整个模型能够学习如何将多个头的注意力结果组合成最终的输出表示。这是模型学习的关键部分。
        # 3. 增加模型表达能力：线性层通过增加可学习参数，提升了模型的表达能力，使其能够更好地捕捉复杂的模式和关系。
        score = score.masked_fill(mask == 0, float('-inf'))
        # 经过softmax然后@v, 点乘之后最后两个维度相乘，又变成(batch, n_head, time, n_d)了
        score = self.softmax(score) @ v

        # 把score的形状变换回来
        score = score.permute(0, 2, 1, 3).contiguous().view(batch, time, dimension)

        # combine score
        # 问题7：这个combine的作用是什么
        output = self.combine(score)
        return output
attention = MultiHeadAttention(d_model, n_head)
output = attention(X, X, X)
print(output, output.shape)

torch.Size([2, 4, 3, 1])
------------
torch.Size([2, 4, 1, 3])
torch.Size([2, 4, 1, 3])
tensor([[[ 0.2031, -0.3362, -0.5117, -0.0695],
         [ 0.1738, -0.2666, -0.5891, -0.0890],
         [ 0.2122, -0.3611, -0.5845, -0.2986]],

        [[ 0.1462, -0.3619, -0.4441,  0.1060],
         [-0.2384, -0.6000, -0.3893,  0.7503],
         [-0.5753, -0.6997, -0.6862,  0.8505]]], grad_fn=<ViewBackward0>) torch.Size([2, 3, 4])


# 第二个版本的MHA

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Optional, List
from labml import tracker

In [55]:
# 线性层+拆分多头
class PrepareForMultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_head: int, d_k: int,  bias: bool = True):
        super().__init__()
        
        self.linear = nn.Linear(d_model, n_head * d_k, bias=bias)
        self.n_head = n_head
        self.d_k = d_k
    
    def forward(self, x: torch.Tensor):
        x = self.linear(x)
        # 拆分最后一层
        head = x.shape[:-1]
        x = x.view(*head, self.n_head, self.d_k)
        return x

In [72]:
# qkv->线性层->qkv拆分->注意力计算、缩放->mask->softmax->drop->注意力分数计算->线性层
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_head: int, drop_out: float = 0.1, bias: bool = True):
        super().__init__()

        # 计算头数量
        self.d_k = d_model // n_head
        self.n_head = n_head
        # 线性层+拆分多头
        self.query = PrepareForMultiHeadAttention(d_model, n_head, self.d_k, bias=bias)
        self.key = PrepareForMultiHeadAttention(d_model, n_head, self.d_k, bias=bias)
        self.value = PrepareForMultiHeadAttention(d_model, n_head, self.d_k, bias=bias)
        self.scores = 0
        # 缩放因子
        self.scale = 1 / math.sqrt(self.d_k)
        # softmax, 对时间维度做softmax
        self.softmax = nn.Softmax(dim=-1)
        # 线性层
        self.output = nn.Linear(d_model, d_model, bias=bias)
        # Dropout
        self.dropout = nn.Dropout(drop_out)
    
    def get_scores(self, query: torch.Tensor, key: torch.Tensor):
        return torch.einsum('ibhd,jbhd->ijbh', query, key)

    def prepareForMask(self, mask: torch.Tensor, query: torch.Tensor, key: torch.Tensor):
        # mask 的形状为[seq_len_q, seq_len_k, batch_size] ，其中第一维是查询维度。如果查询维度等于1，则会进行广播。
        assert mask[0] == 1 or mask[0] == query.shape[0]
        assert mask[1] == key.shape[0]
        assert mask[2] == 1 or mask[2] == query.shape[1]
        mask = mask.unsqueeze(-1)
        return mask

    # 问题： 这个*是什么意思
    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, mask: Optional[torch.Tensor] = None):
        # 计算注意力权重
        # 变换之后query的形状为(seq_len, batch_size, n_head, d_k)
        seq_len, batch_size, _ = query.shape
        # 准备掩码
        if mask is not None:
            mask = self.prepare_mask(mask, query.shape, key.shape)

        query = self.query(query)
        key = self.key(key)
        value = self.value(value)
        
        scores = self.get_scores(query, key)
        # 缩放
        scores = scores * self.scale
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        attn = self.softmax(scores)
        # 调试保存注意力信息
        # tracker.debug('attn', attn)
        attn = self.dropout(attn)

        # 计算注意力
        # 问题： 爱因斯坦求和的过程是怎样的
        x = torch.einsum('ijbh,jbhd->ibhd', attn, value)
        # 问题： 这句话的作用是什么
        self.attn = attn.detach()
        x = x.reshape(seq_len, batch_size, -1)

        # 最后加入线性层
        return self.output(x)

In [73]:
# 定义超参数
seq_len = 1
batch_size = 2
d_model = 6
n_head = 2
X = torch.randn(seq_len, batch_size, d_model)

multiHeadAttention = MultiHeadAttention(d_model, n_head)
output = multiHeadAttention(X, X, X)